### 1. "Телышев" - Вариант №2

In [1]:
surname = "Телышев"  # Ваша фамилия

alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
variants = [4, 42, 21, 21, 34,  1, 44, 26, 18, 43, 38, 26, 18, 43,  3, 49, 45,
            7, 42, 25,  4,  9, 36, 33, 31, 29,  5, 31,  4, 19, 24, 27, 33]
d = dict(zip(alphabet, variants))
variant =  sum([d[element] for element in surname.lower()]) % 2 + 1
print("Ваш вариант - ", variant)

Ваш вариант -  2


### 2. Рейтинги

#### Задача 1. Средний рейтинг
```
Реализуйте подсчет среднего рейтинга продуктов. Результат сохранить в HDFS в файле "avg_rating.csv".
Формат каждой записи: ProdId,Rating
```

In [2]:
rdd_products = sc.textFile('/user/cloudera/hw_part_1/spark/task_1/data/samples_100.json')
# rdd_products = sc.textFile('/user/cloudera/hw_part_1/spark/task_1/data/Electronics.json')

In [5]:
import json

def get_product_rating(product_json):
    product = json.loads(product_json)
    return (product.get('asin'), product.get('overall'))

rdd_product_rating = rdd_products.map(lambda product_json: get_product_rating(product_json))
rdd_product_rating.take(10)

[('0528881469', 5.0),
 ('0528881469', 1.0),
 ('0528881469', 3.0),
 ('0528881469', 2.0),
 ('0528881469', 1.0),
 ('0594451647', 5.0),
 ('0594451647', 2.0),
 ('0594451647', 5.0),
 ('0594451647', 4.0),
 ('0594451647', 5.0)]

In [15]:
rdd_product_avg_rating = (
    rdd_product_rating
        .aggregateByKey(
                (0, 0),
                lambda sum_count, rating: (sum_count[0] + rating, sum_count[1] + 1),
                lambda sum_count_x, sum_count_y: (sum_count_x[0] + sum_count_y[0],
                                                  sum_count_x[1] + sum_count_y[1])
            )
        .mapValues(lambda sum_count: sum_count[0] / sum_count[1])
)
print(rdd_product_avg_rating.collect())

def to_csv_line(data):
    delimiter = ','
    fields = []
    for item in data:
        field = str(item)
        if delimiter in field:
            field = '"{}"'.format(field)
        fields.append(field)
    return delimiter.join(fields)

print(rdd_product_avg_rating.map(to_csv_line).collect())

[('0972683275', 4.390243902439025), ('0528881469', 2.4), ('0594451647', 4.2), ('0594481813', 4.0)]
['0972683275,4.390243902439025', '0528881469,2.4', '0594451647,4.2', '0594481813,4.0']


```python
# product_avg_rating.py
import json
import sys
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('ProductAvgRating')
sc = SparkContext(conf=conf)

rdd_products = sc.textFile(sys.argv[1])

def get_product_rating(product_json):
    product = json.loads(product_json)
    return (product.get('asin'), product.get('overall'))

rdd_product_rating = rdd_products.map(lambda product_json: get_product_rating(product_json))
rdd_product_avg_rating = (
    rdd_product_rating
        .aggregateByKey(
                (0, 0),
                lambda sum_count, rating: (sum_count[0] + rating, sum_count[1] + 1),
                lambda sum_count_x, sum_count_y: (sum_count_x[0] + sum_count_y[0],
                                                  sum_count_x[1] + sum_count_y[1])
            )
        .mapValues(lambda sum_count: sum_count[0] / sum_count[1])
)

def to_csv_line(data):
    delimiter = ','
    fields = []
    for item in data:
        field = str(item)
        if delimiter in field:
            field = '"{}"'.format(field)
        fields.append(field)
    return delimiter.join(fields)

rdd_product_avg_rating.map(to_csv_line).saveAsTextFile(sys.argv[2])
```

#### Задача 2. Добавление наименования продукта
```
Напишите программу, которая каждому ProdId из "avg_rating.csv" ставит в соответстие названием продукта.
Результат сохранить в HDFS в файле "prodname_avg_rating.csv": ProdId,Name,Rating
```

In [ ]:
hdfs dfs -rm -r -f -skipTrash /user/cloudera/hw_part_1/spark/task_2
hdfs dfs -mkdir -p /user/cloudera/hw_part_1/spark/task_2/data

hdfs dfs -copyFromLocal /home/cloudera/Desktop/data_samples/avg_rating.csv \
                        /user/cloudera/hw_part_1/spark/task_2/data/avg_rating.csv

hdfs dfs -copyFromLocal /home/cloudera/Desktop/data_samples/samples_100_meta.json \
                        /user/cloudera/hw_part_1/spark/task_2/data/samples_100_meta.json

# hdfs dfs -copyFromLocal /home/cloudera/Desktop/data_samples/Electronics_meta.json \
#                         /user/cloudera/hw_part_1/spark/task_2/data/Electronics_meta.json

hdfs dfs -ls -R /user/cloudera/hw_part_1/spark/task_2/